In [2]:
import os
import re
import time
import pickle

import nd2reader

import pickle

import pandas as pd
import numpy as np

from scipy.spatial import distance
from tifffile import imread,imsave
from skimage import transform
from skimage.transform import downscale_local_mean

import matplotlib.pyplot as plt

import napari

from pystackreg import StackReg
import pystackreg

import mxnet
from cellpose import models

import ipywidgets as widgets
from IPython.display import display

creating new log file
2021-07-25 17:31:50,269 [INFO] WRITING LOG OUTPUT TO C:\Users\kmkedz\.cellpose\run.log


In [4]:
pathData=r'Z:\Wayne\20210716_sen_sig_IF'

myWell = 'A3'

## Read in segmented masks

In [17]:
# read in segmentation results

myLabels = []

for my_file in [x for x in os.listdir(os.path.join(pathData,'segmented')) if myWell in x]:
    
    print(my_file)
    
    temp = imread(os.path.join(pathData,'segmented',my_file))
    
    myLabels.append(temp)
    
myLabels = np.array(myLabels)

210622_well_A3_0000_label.tif
210622_well_A3_0001_label.tif
210622_well_A3_0002_label.tif


In [18]:
myLabels.shape

(3, 16590, 16589)

## Registration

In [19]:
downscale_factor = 4

labels_small = myLabels>0
labels_small = downscale_local_mean(labels_small,(1,downscale_factor,downscale_factor))

In [20]:
labels_small.shape

(3, 4148, 4148)

In [21]:
%%time

name = 'RIGID_BODY'
tf = StackReg.RIGID_BODY
reference = 'first'

sr = StackReg(tf)

tmat_small = sr.register_stack(labels_small, axis=0, reference=reference, verbose=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:48<00:00, 24.29s/it]

Wall time: 50.1 s


In [22]:
%%time

# apply transformations to check if they were all successful

res = []

for index,tranform_matrix in enumerate(tmat_small):
    
    eu_transform_small = transform.EuclideanTransform(tranform_matrix)
    
    # if you want to check only transformation without rotation
    #eu_transform_small = transform.EuclideanTransform(translation = eu_transform_small.translation, rotation = 0)
    
    temp = transform.warp(labels_small[index,:,:],eu_transform_small,output_shape=labels_small[index].shape)
    
    res.append(temp)
    
res = np.array(res)

Wall time: 2.93 s


In [34]:
viewer = napari.Viewer()

In [28]:
viewer = napari.Viewer()
viewer.add_image(res[0],blending ='additive')
viewer.add_image(res,blending ='additive',colormap='green')

<Image layer 'res' at 0x25873022708>

In [14]:
#viewer.add_image(labels_small,blending ='additive',colormap='red')

In [30]:
# transform results of stackreg to work as Euclidean transform on original size images

tmat =[]

for tranform_matrix in tmat_small:
    
    eu_transform_small = transform.EuclideanTransform(tranform_matrix)
    
    eu_transform = transform.EuclideanTransform(translation = eu_transform_small.translation * downscale_factor,
                                                rotation = eu_transform_small.rotation)
    
    tmat.append(eu_transform)

In [31]:
saveDir = os.path.join(pathData,'transforms')